In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import glob
import os
from math import log

In [3]:
allFiles = sorted(glob.glob("*.txt"),key=os.path.getmtime) # includes all files into a single stucture,
# Sorted by modification time so that the first measurement is first...
print(allFiles)

['PMT_Background_280Deg_07112017.txt', 'PMT_Signal_280Deg_07112017.txt', 'PMT_Background_275Deg_07112017.txt', 'PMT_Signal_275Deg_07112017.txt', 'PMT_Background_270Deg_07112017.txt', 'PMT_Signal_270Deg_07112017.txt', 'PMT_Background_265Deg_07112017.txt', 'PMT_Signal_265Deg_07112017.txt', 'PMT_Background_260Deg_07112017.txt', 'PMT_Signal_260Deg_07112017.txt', 'PMT_LaserBackground_07112017.txt']


In [4]:
frame = pd.DataFrame()
df = pd.DataFrame()

In [5]:
for i,file_ in enumerate(allFiles):
#    f = open(file_,'r')
#     for j,line in enumerate(f):
#         if j == 1:
#             date = line[14:22]
#             break
    df = pd.read_csv(file_,index_col = None, header=None, skiprows = 10, skipfooter = 1, delimiter = '\t')
#     frame[file_] = df[5]
    df['file']=file_
    df['timeDelay']= df[1]
    df['PDvalue'] = df[3]
#     df['date']=date
    if i < 1:
        frame = df
    else:
        frame = frame.append(df)

/Users/sanjaykhatri/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skip_footer; you can avoid this warning by specifying engine='python'.


In [6]:
c = 3*10**11; #Speed of Light in mm/s, Going to use this to convert spatial information to time. 
T1 = (417*0.000125 + 0.0229)*2*25.4/(10**-12*c)
def ConvertToTimeDelay(x,itr,offset):
    c = 3*10**11
    timeDelay2 = []
    for i in range(0,len(x)):
        timedelay = -((x[i,0]*itr + offset)*25.4*2/(10**-12*c))+T1
        timeDelay2.append(timedelay)
    return(timeDelay2)

In [11]:
times = pd.Series()
PDvalues = pd.Series()
AvgPD50 = pd.Series()
itr = 0.000125
offset = 0.023
for i in range(0,200):
    time = frame[1][i]
    PDvalue = frame[3][i]
    times = times.append(time)
    PDvalues = PDvalues.append(PDvalue)
#print(len(PDvalues))
    
times = times.values.reshape(200,len(allFiles))
PDvalues = PDvalues.values.reshape(200,len(allFiles))
Background1 = PDvalues[:,0]
Background2 = PDvalues[:,2]
Background3 = PDvalues[:,4]
Background4 = PDvalues[:,6]
Background5 = PDvalues[:,8]
PMT_Signal1 = PDvalues[:,1]
PMT_Signal2 = PDvalues[:,3]
PMT_Signal3 = PDvalues[:,5]
PMT_Signal4 = PDvalues[:,7]
PMT_Signal5 = PDvalues[:,9]
SHGBackground = PDvalues[:,10]

MeanBackground1 = np.mean(Background1)
MeanBackground2 = np.mean(Background2)
MeanBackground3 = np.mean(Background3)
MeanBackground4 = np.mean(Background4)
MeanBackground5 = np.mean(Background5)
MeanSHG = np.mean(SHGBackground)

Real_Signal1 = PMT_Signal1-MeanBackground1-MeanSHG
Real_Signal2 = PMT_Signal2-MeanBackground2-MeanSHG
Real_Signal3 = PMT_Signal3-MeanBackground3-MeanSHG
Real_Signal4 = PMT_Signal4-MeanBackground4-MeanSHG
Real_Signal5 = PMT_Signal5-MeanBackground5-MeanSHG

Mean_PMT1 = np.mean(Real_Signal1)
Mean_PMT2 = np.mean(Real_Signal2)
Mean_PMT3 = np.mean(Real_Signal3)
Mean_PMT4 = np.mean(Real_Signal4)
Mean_PMT5 = np.mean(Real_Signal5)


Mean_PMT=np.array([Mean_PMT1,Mean_PMT2,Mean_PMT3,Mean_PMT4,Mean_PMT5])
THzEnergy = np.array([470,450,400,330,245])
print(Mean_PMT[0])
print(MeanSHG)

THzSHGEfficiency = Mean_PMT1/MeanSHG
print(THzSHGEfficiency)
LogPMT = np.log10(np.absolute(Mean_PMT))
LogTHz = np.log10(THzEnergy)
STD_PMT = Mean_PMT.std(axis = 0)
err1 = STD_PMT/(200**0.5)

w=(err1/(np.absolute(Mean_PMT)*log(10)))**-2
Delta = sum(w)*sum(w*LogTHz**2) - (sum(w*LogTHz))**2
A = (sum(w*LogTHz**2)*sum(w*LogPMT) - sum(w*LogTHz)*sum(w*LogTHz*LogPMT))/Delta
B = (sum(w)*sum(w*LogTHz*LogPMT) - (sum(w*LogPMT)*sum(w*LogTHz)))/Delta
# Least Squares Fitting Parameters above
print(w)
print(A)
print(B)
# Errors in Least Squares Plotting
err = (((len(LogPMT)-2)**-1) * sum((LogPMT - A -B*LogTHz)**2)**0.5)
errorA = (sum(w*LogTHz**2)/Delta)**0.5
errorB = (sum(w)/Delta)**0.5
print(errorA)
print(errorB)


plt.plot(LogTHz, A + B*LogTHz,'r',label = 'Fit: y=0.81x -3.43')
plt.plot(LogTHz, LogPMT,'b.', label = 'Data')
plt.grid(True)
plt.xlabel('Log(THz Energy)')
plt.ylabel('Log(PMT Signal)')
plt.legend(loc = 'upper left')
plt.show()
#plt.savefig('PMT_SignalVsTHzEnergy.png')




-0.0536565
-0.1349525
0.397595450251
[ 43543.55438112  41688.3155389   34657.47979209  27296.66256589
  14214.72636473]
-3.43546335034
0.81245948632
0.0765035557717
0.0293943043643


In [9]:
fit = np.polyfit(LogTHz,LogPMT,1,w=err1/(np.absolute(Mean_PMT)*log(10)))
f = np.poly1d(fit)
pPower = np.linspace(LogTHz[1],LogTHz[-1],100)
f_new = f(pPower)
print(f_new)
print(f)

[-1.27524472 -1.27758936 -1.27993401 -1.28227865 -1.2846233  -1.28696794
 -1.28931259 -1.29165723 -1.29400188 -1.29634652 -1.29869117 -1.30103581
 -1.30338046 -1.3057251  -1.30806975 -1.31041439 -1.31275904 -1.31510368
 -1.31744833 -1.31979297 -1.32213762 -1.32448226 -1.32682691 -1.32917155
 -1.3315162  -1.33386084 -1.33620549 -1.33855013 -1.34089478 -1.34323942
 -1.34558407 -1.34792871 -1.35027336 -1.352618   -1.35496265 -1.35730729
 -1.35965194 -1.36199658 -1.36434123 -1.36668587 -1.36903052 -1.37137516
 -1.37371981 -1.37606445 -1.3784091  -1.38075374 -1.38309839 -1.38544303
 -1.38778768 -1.39013232 -1.39247697 -1.39482161 -1.39716626 -1.3995109
 -1.40185555 -1.40420019 -1.40654484 -1.40888948 -1.41123413 -1.41357877
 -1.41592342 -1.41826806 -1.42061271 -1.42295735 -1.425302   -1.42764664
 -1.42999129 -1.43233593 -1.43468058 -1.43702522 -1.43936987 -1.44171451
 -1.44405916 -1.4464038  -1.44874845 -1.45109309 -1.45343774 -1.45578238
 -1.45812703 -1.46047167 -1.46281632 -1.46516096 -1.